# The API: Connecting to BigML

The BigML API offers an endpoint to create, get, update and delete every Machine Learning resource. <br/>
This API is accessible via HTTP and its general public domain is [bigml.io](https://bigml.io). <br/>
You will need some credentials that will be used to authenticate every request. <br/>
We recommend to set them in environment variables for your convenience.

In [4]:
# set your credentials as environment variables
%env BIGML_USERNAME=merce_demo
%env BIGML_API_KEY=9995b2f705b72de0c6ca90f1e1446acbf35daea8

env: BIGML_USERNAME=merce_demo
env: BIGML_API_KEY=9995b2f705b72de0c6ca90f1e1446acbf35daea8


The API calls that you will need to issue contain these credentials as authentication token. For instance

In [5]:
url = "https://bigml.io/source?username=merce_demo;api_key=9995b2f705b72de0c6ca90f1e1446acbf35daea8;limit=1"
print(url)

https://bigml.io/source?username=merce_demo;api_key=9995b2f705b72de0c6ca90f1e1446acbf35daea8;limit=1


In [6]:
!curl "https://bigml.io/source?username=$BIGML_USERNAME;api_key=$BIGML_API_KEY;limit=1"

{"meta": {"limit": 1, "next": "/andromeda/source?username=merce_demo&api_key=9995b2f705b72de0c6ca90f1e1446acbf35daea8&limit=1&offset=1", "offset": 0, "previous": null, "total_count": 352}, "objects": [{"category": 12, "charset": "UTF-8", "code": 200, "configuration": null, "configuration_status": false, "content_type": "text/csv", "created": "2019-03-06T23:24:32.657000", "creator": "merce_demo", "credits": 0, "description": "Created using BigMLer", "disable_datetime": false, "field_types": {"categorical": 1, "datetime": 0, "items": 0, "numeric": 8, "text": 0, "total": 9}, "fields_meta": {"count": 9, "limit": 200, "offset": 0, "query_total": 9, "total": 9}, "file_name": "diabetes_more.csv", "item_analysis": {}, "md5": "460be5cd506263cbac1331db14049f91", "name": "Cumulative diabetes data", "name_options": "9 fields (1 categorical, 8 numeric)", "number_of_anomalies": 0, "number_of_anomalyscores": 0, "number_of_associations": 0, "number_of_associationsets": 0, "number_of_centroids": 0, "nu

lists the last previously uploaded file in your account.<br/>
Managing resources using these raw HTTP calls is, of course, possible but not optimal.<br/>
Bindings to several languages will make easier resource management.

## The bindings: connecting to BigML

From now on, this notebook uses the **Python bindings library** and **BigMLer**, a command line utility, to access the BigML API. 
Please, check the **quick start** section of [BigMLer's documentation](http://bigmler.readthedocs.org/en/latest/#quick-start) to know how to **install** and remember to [set your **credentials**](http://bigml.readthedocs.org/en/latest/#authentication) before using **BigMLer** or the bindings.

In [7]:
from bigml.api import BigML
api = BigML() # Credentials are imported from environment variables
              # BIGML_USERNAME and BIGML_API_KEY.
              # They can also be set explicitly: api = BigML([username], [api_key])
print(api.url + api.auth)

https://bigml.io/andromeda/?username=merce_demo;api_key=9995b2f705b72de0c6ca90f1e1446acbf35daea8;


The next instruction should show the last uploaded source in your account. If that's the case, your credentials are correctly set and you are ready to start creating your resources in BigML

In [8]:
api.list_sources("limit=1")

{'code': 200,
 'error': None,
 'meta': {'limit': 1,
  'next': '/andromeda/source?username=merce_demo&api_key=9995b2f705b72de0c6ca90f1e1446acbf35daea8&limit=1&offset=1',
  'offset': 0,
  'previous': None,
  'total_count': 352},
 'objects': [{'category': 12,
   'charset': 'UTF-8',
   'code': 200,
   'configuration': None,
   'configuration_status': False,
   'content_type': 'text/csv',
   'created': '2019-03-06T23:24:32.657000',
   'creator': 'merce_demo',
   'credits': 0,
   'description': 'Created using BigMLer',
   'disable_datetime': False,
   'field_types': {'categorical': 1,
    'datetime': 0,
    'items': 0,
    'numeric': 8,
    'text': 0,
    'total': 9},
   'fields_meta': {'count': 9,
    'limit': 200,
    'offset': 0,
    'query_total': 9,
    'total': 9},
   'file_name': 'diabetes_more.csv',
   'item_analysis': {},
   'md5': '460be5cd506263cbac1331db14049f91',
   'name': 'Cumulative diabetes data',
   'name_options': '9 fields (1 categorical, 8 numeric)',
   'number_of_anomal

### PREDICTION WORKFLOW

In [9]:
import pandas as pd
import sys
from pprint import pprint, pformat
from IPython.display import display
# DIABETES_FILE = 'https://static.bigml.com/csv/diabetes.csv' # you can also import from a remote file
DIABETES_FILE = 'data/diabetes.csv'
display(pd.read_csv(DIABETES_FILE, nrows=5))

,pregnancies,plasma glucose,blood pressure,triceps skin thickness,insulin,bmi,diabetes pedigree,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,True
1,1,85,66,29,0,26.6,0.351,31,False
2,8,183,64,0,0,23.3,0.672,32,True
3,1,89,66,23,94,28.1,0.167,21,False
4,0,137,40,35,168,43.1,2.288,33,True


Let's upload this file and see how this content is interpreted from the Machine Learning point of view.<br/>
We'll first create a **Project**, an organizational unit to store every resource generated in this session.

#### CREATING PROJECT

In [10]:
PROJECT_NAME = "MLSEV Python bindings example"
project = api.create_project({'name': PROJECT_NAME})

**Projects** and **predictions** are the only resources that are **synchronous** in **BigML**, meaning that when you issue the create call the response you get is never a work in process, but the final resource.

In [11]:
pprint(project)

{'code': 201,
 'error': None,
 'location': 'http://bigml.io/andromeda/project/5c815181c7736e3b48008aac',
 'object': {'category': 0,
            'code': 201,
            'configuration': None,
            'configuration_status': False,
            'created': '2019-03-07T17:14:41.063983',
            'creator': 'merce_demo',
            'description': '',
            'execution_id': None,
            'execution_status': None,
            'manage_permission': [],
            'name': 'MLSEV Python bindings example',
            'name_options': '',
            'private': True,
            'resource': 'project/5c815181c7736e3b48008aac',
            'stats': {'anomalies': {'count': 0},
                      'anomalyscores': {'count': 0},
                      'associations': {'count': 0},
                      'associationsets': {'count': 0},
                      'batchanomalyscores': {'count': 0},
                      'batchcentroids': {'count': 0},
                      'batchpredictions'

The first level attributes of this dictionary contain:

- code: the HTTP response status code
- error: the error information (when an HTTP error occurs)
- location: the location to access the resource
- object: the API's response
- resource: the **resource ID**

In [12]:
PROJECT = project['resource']

The rest of resources in **BigML** are **asynchronous**, so you will need polling for the resource till it is either finished or faulty. We'll see the first example now, when we upload our data to the platform and create a **source**.

#### CREATING SOURCE
When data is uploaded to the platform a **source** is created

In [13]:
source = api.create_source(DIABETES_FILE,
                           {'name': 'diabetes source', \
                            'tags': ['bindings example', 'diabetes'], \
                            'project': PROJECT})
"""
    CSV, ARFF, Excel and JSON files, either local or remote, can be uploaded.
    For instance, you could use a remote diabetes:
    DIABETES_FILE = "https://static.bigml.com/csv/diabetes.csv"
    
"""
pprint(source)

{'code': 201,
 'error': None,
 'location': 'http://bigml.io/andromeda/source/5c815231c7736e3b48008abb',
 'object': {'category': 0,
            'code': 201,
            'configuration': None,
            'configuration_status': False,
            'content_type': 'text/csv',
            'created': '2019-03-07T17:17:37.991396',
            'creator': 'merce_demo',
            'credits': 0.0,
            'description': '',
            'disable_datetime': False,
            'field_types': {'categorical': 0,
                            'datetime': 0,
                            'items': 0,
                            'numeric': 0,
                            'text': 0,
                            'total': 0},
            'fields_meta': {'count': 0, 'limit': 1000, 'offset': 0, 'total': 0},
            'file_name': 'diabetes.csv',
            'item_analysis': {},
            'md5': '460be5cd506263cbac1331db14049f91',
            'name': 'diabetes source',
            'name_options': '',
      

As you can see, this response does not contain any of the uploaded information yet. The **status** of the resource shows that the source creation request is in process. We'll have to wait for this process to finish. This is what **api.ok** does

In [14]:
api.ok(source)
pprint(source["object"])

{'category': 0,
 'charset': 'UTF-8',
 'code': 200,
 'configuration': None,
 'configuration_status': False,
 'content_type': 'text/csv',
 'created': '2019-03-07T17:17:37.991000',
 'creator': 'merce_demo',
 'credits': 0,
 'description': '',
 'disable_datetime': False,
 'field_types': {'categorical': 1,
                 'datetime': 0,
                 'items': 0,
                 'numeric': 8,
                 'text': 0,
                 'total': 9},
 'fields': {'000000': {'column_number': 0,
                       'name': 'pregnancies',
                       'optype': 'numeric',
                       'order': 0},
            '000001': {'column_number': 1,
                       'name': 'plasma glucose',
                       'optype': 'numeric',
                       'order': 1},
            '000002': {'column_number': 2,
                       'name': 'blood pressure',
                       'optype': 'numeric',
                       'order': 2},
            '000003': {'column_numb

**api.ok** waits for the source creation to finish and updates the contents of the **source** variable with the current remote version of the source. Thus, now we can see that the **source** variable contains the description of the fields inferred from the uploaded file. We'll write two auxiliar functions using **api.ok** to show the resources once they are finished or to warn us about any errors.

In [15]:
def check(resource):
    """
        Checks whether the resource status is *finished* or
        prints an error if something fails.
    """
    # api.ok uses api.get_[resouce_type] to retrieve the status of the resource
    # till it reaches a final state (either FINISHED or FAULTY)
    # as defined in 
    if not api.ok(resource):
        print("Error!!!: Failed to create resource %s" % \
            resource.get(
                'resource',
                resource.get('object', {}).get('name')))

def check_and_show(resource):
    """
        Checks whether the resource status is *finished*
        and shows its contents or prints an error if something failed.
    """
    check(resource)
    pprint(resource)


#### View source in BIGML's web site
As all **BigML**'s applications work on top of the same **API**, the source we've just created appears immediately in the source listings of our web dashboard.

In [16]:
BIGML_DASHBOARD_URL = 'https://bigml.com/dashboard'
sources_list_url = "%s/sources" % BIGML_DASHBOARD_URL
print(sources_list_url)

https://bigml.com/dashboard/sources


Now that our data is uploaded, we'd need to check that the **fields** characteristics inferred in our **source** are really the expected ones. As that's the case, the next step will be creating a **dataset** from it.

#### CREATING DATASET

The dataset will provide information about **errors**, **missing** values in fields and **histograms**.

In [17]:
dataset = api.create_dataset(source)
check(dataset)

Summaries show the number of **missings** and **errors** and we can decide what to do with them.

#### CREATING MODEL
This is the real training part, where the algorithm learns the patterns in your data. Un supervised learning models, the **objective field** is the label that will be predicted by the model. BigML considers the last field to be the objective field if not said otherwise.

In [18]:
model = api.create_model( \
    dataset,
    {'name': "Diabetes decision tree",
     'objective_field': "diabetes"})
check(model)

## PREDICTIONS INTEGRATION
Eventually, the goal of our models will usually be creating predictions. We can create **remote predictions** by providing the new input data.

In [19]:
input_data = {'plasma glucose': 180, 'bmi': 30}
prediction = api.create_prediction(model,
                                   input_data=input_data)
check(prediction)
print("prediction: %s" % prediction["object"]["output"])
print("confidence: %s" % prediction["object"]["confidence"])
print("path: %s" % prediction["object"]["prediction_path"]["path"])

prediction: true
confidence: 0.79758
path: [{'field': '000001', 'operator': '>', 'value': 123}, {'field': '000001', 'operator': '>', 'value': 166}, {'field': '000005', 'operator': '>', 'value': 23.1}]


Of course, this method has latencies involved every time you make a prediction. If your predictions don't need to be immediate, then you can store the input data in a file and do a **remote batch prediction** with an entire test dataset.

## Model class: using the model locally to predict
The JSON model that can be downloaded via the API has all the information needed to predict.

In [20]:
LOCAL_MODEL_FILE = "data/diabetes_model.json"
api.export(model, LOCAL_MODEL_FILE)

'data/diabetes_model.json'

 The local **Model** object adds a **predict** method that can be used locally.

In [21]:
from bigml.model import Model
"""
    The **Model** object can use the contents of a Model
    previously stored in a file or
    internally download the model JSON structure once and
    store it in a local directory for further use.
"""
local_model = Model(LOCAL_MODEL_FILE)
pprint(local_model.predict(input_data, full=True))

{'confidence': 0.79758,
 'count': 24,
 'distribution': [['true', 23], ['false', 1]],
 'distribution_unit': 'categories',
 'next': 'blood pressure',
 'path': ['plasma glucose > 123', 'plasma glucose > 166', 'bmi > 23.1'],
 'prediction': 'true',
 'probability': 0.9362}


If you need to predict many rows at once, you can use the **BigMLer** command line, that uses this local **Model** object to create the predictions and store it in a file.

## Workflows

#### Basic prediction workflow

To sum up, the basic prediction workflow will need some steps:

- Upload the data to create a Source
- Summarize all data in a Dataset
- Create a Model from the Dataset
- Use the Model to produce a prediction for the new data

Using the diabetes example, to produce this workflow using the bindings you would use this code

In [ ]:
import csv
from bigml.api import BigML
from bigml.model import Model

api = BigML()
source = api.create_source(DIABETES_FILE, {"project": PROJECT})
api.ok(source)
dataset = api.create_dataset(source)
api.ok(dataset)
model = api.create_model(dataset)
api.ok(model)

local_model = Model(model)
with open("data/diabetes_test.csv") as test_handler:
    reader = csv.DictReader(test_handler)
    for input_data in reader:
    # predicting for all rows
        print(local_model.predict(input_data))

**Back to the slides** to learn how this can be improved

#### Canonical Workflows: BigMLer

The same could be achieved in a single line command

In [ ]:
DIABETES = "data/diabetes.csv"
DIABETES_TEST = "data/diabetes_test.csv"
!bigmler --train $DIABETES --test $DIABETES_TEST \
         --output-dir diabetes-prediction \
         --project-id $PROJECT \
         --tag "best-diabetes" \
         --name "Diabetes with BigMLer"

And if we want to evaluate this model, we can add the **--evaluate** flag

In [ ]:
!bigmler --train $DIABETES \
         --test-split 0.2 \
         --output-dir diabetes-eval \
         --project-id $PROJECT \
         --name "Diabetes evaluated with BigMLer" \
         --seed "bigml" \
         --evaluate


#### Retrain with cumulative data

Usually, you start your project uploading a sample of data and playing with it till you discover the workflow that gives you acceptable results. Then, the rest of data is uploaded and you'd like to repeat the same process on the accumulated data. **BigMLer** can help you do that. In this example, we do a regular model creation workflow.

In [ ]:
!bigmler --version

In [ ]:
!bigmler --train data/diabetes.csv \
         --tag best_diabetes \
         --name "Cumulative diabetes data" \
         --output-dir ./initial_model

In [ ]:
MODEL_ID = "model/5c80070d7811dd1853006c04"
!bigmler retrain --add data/diabetes_more.csv \
                 --id $MODEL_ID \
                 --output-dir cumulative_retrain

In [22]:
from bigml.api import BigML
api = BigML()

In [23]:
api.create_source("/home/merce/Downloads/chat2.csv")

{'code': 201,
 'error': None,
 'location': 'http://bigml.io/andromeda/source/5c881cd85299633066001007',
 'object': {'category': 0,
  'code': 201,
  'configuration': None,
  'configuration_status': False,
  'content_type': 'text/csv',
  'created': '2019-03-12T20:55:52.348306',
  'creator': 'merce_demo',
  'credits': 0.0,
  'description': '',
  'disable_datetime': False,
  'field_types': {'categorical': 0,
   'datetime': 0,
   'items': 0,
   'numeric': 0,
   'text': 0,
   'total': 0},
  'fields_meta': {'count': 0, 'limit': 1000, 'offset': 0, 'total': 0},
  'file_name': 'chat2.csv',
  'item_analysis': {},
  'md5': '6f4f1565c869c5839e86e54caff5350c',
  'name': 'chat2.csv',
  'name_options': '',
  'number_of_anomalies': 0,
  'number_of_anomalyscores': 0,
  'number_of_associations': 0,
  'number_of_associationsets': 0,
  'number_of_centroids': 0,
  'number_of_clusters': 0,
  'number_of_correlations': 0,
  'number_of_datasets': 0,
  'number_of_deepnets': 0,
  'number_of_ensembles': 0,
  'numb

In [24]:
api.auth

'?username=merce_demo;api_key=9995b2f705b72de0c6ca90f1e1446acbf35daea8;'